hitter projections

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,r2_score
import ipywidgets as widgets
from IPython.display import display,clear_output

TARGET_SEASON=2026
WINDOW_N_FOR_LEADERBOARD=50
MIN_PA_FILTER_DEFAULT=0
MIN_ROWS_TO_TRAIN=50

def to_scalar(x):
    if isinstance(x,pd.Series): x=x.iloc[0]
    x=pd.to_numeric(x,errors="coerce")
    return float(x) if pd.notna(x) else np.nan

def safe_div(num,den):
    num=pd.to_numeric(num,errors="coerce")
    den=pd.to_numeric(den,errors="coerce")
    return np.where((den>0)&np.isfinite(den)&np.isfinite(num),num/den,np.nan)

def build_name(df):
    if "last_name, first_name" in df.columns:
        df["Name"]=df["last_name, first_name"].astype(str)
        return df
    if "last_name" in df.columns and "first_name" in df.columns:
        df["Name"]=df["last_name"].astype(str)+", "+df["first_name"].astype(str)
        return df
    if "Name" in df.columns:
        return df
    raise ValueError("Missing name column. Need 'last_name, first_name' or Name or (first_name+last_name).")

def map_hitter_columns(df_in):
    d=df_in.copy()
    rename={
        "player_age":"age",
        "hit":"H",
        "single":"1B",
        "double":"2B",
        "triple":"3B",
        "home_run":"HR",
        "strikeout":"K",
        "walk":"BB",
        "batting_avg":"BA_csv",
        "slg_percent":"SLG_csv",
        "on_base_percent":"OBP_csv",
        "on_base_plus_slg":"OPS_csv",
        "b_rbi":"RBI",
        "b_total_bases":"TB_csv",
        "r_total_caught_stealing":"CS",
        "r_total_stolen_base":"SB",
        "b_gnd_into_dp":"GIDP",
        "b_sac_fly":"SF",
        "r_run":"R",
    }
    d=d.rename(columns={k:v for k,v in rename.items() if k in d.columns})
    return d

def compute_rates_from_projected_counts(p):
    ab=to_scalar(p.get("ab",np.nan))
    H=to_scalar(p.get("H",np.nan))
    BB=to_scalar(p.get("BB",np.nan))
    _1B=to_scalar(p.get("1B",np.nan))
    _2B=to_scalar(p.get("2B",np.nan))
    _3B=to_scalar(p.get("3B",np.nan))
    HR=to_scalar(p.get("HR",np.nan))
    TB=to_scalar(p.get("TB",np.nan))
    if (not np.isfinite(TB)) and all(np.isfinite(x) for x in [_1B,_2B,_3B,HR]):
        TB=_1B+2*_2B+3*_3B+4*HR
    BA=H/ab if np.isfinite(H) and np.isfinite(ab) and ab>0 else np.nan
    OBP=(H+BB)/(ab+BB) if np.isfinite(H) and np.isfinite(BB) and np.isfinite(ab) and (ab+BB)>0 else np.nan
    SLG=TB/ab if np.isfinite(TB) and np.isfinite(ab) and ab>0 else np.nan
    OPS=OBP+SLG if np.isfinite(OBP) and np.isfinite(SLG) else np.nan
    return {"BA":BA,"OBP":OBP,"SLG":SLG,"OPS":OPS,"TB":TB}

def build_X_pred(model,stat_name,current,age_now,pa_val,age_delta):
    cols=list(model.feature_names_in_)
    row={}
    for c in cols:
        if c==stat_name: row[c]=current
        elif c=="age": row[c]=age_now
        elif c=="pa": row[c]=pa_val
        elif c==f"{stat_name}_age_delta": row[c]=age_delta
        else: row[c]=np.nan
    return pd.DataFrame([row],columns=cols)

def ensure_1b_tb(d):
    if ("1B" not in d) and all(k in d for k in ["H","2B","3B","HR"]):
        v=to_scalar(d["H"])-to_scalar(d["2B"])-to_scalar(d["3B"])-to_scalar(d["HR"])
        d["1B"]=max(0.0,v) if np.isfinite(v) else np.nan
    if ("TB" not in d) and all(k in d for k in ["1B","2B","3B","HR"]):
        d["TB"]=to_scalar(d["1B"])+2*to_scalar(d["2B"])+3*to_scalar(d["3B"])+4*to_scalar(d["HR"])
    return d

def format_triple(b,o,s):
    if np.isfinite(b) and np.isfinite(o) and np.isfinite(s):
        return f"{b:.3f}/{o:.3f}/{s:.3f}"
    return ""

def fmt_cell(x):
    if isinstance(x,str): return x
    if pd.isna(x): return ""
    try:
        return f"{float(x):.4f}"
    except Exception:
        return str(x)

df=pd.read_csv("stats (31).csv").copy()
df=df.loc[:,~df.columns.duplicated()].copy()
df=build_name(df)
df=map_hitter_columns(df)

required={"Name","player_id","year","age","pa","H","1B","2B","3B","HR","BB","BA_csv"}
missing=required-set(df.columns)
if missing:
    raise ValueError(f"Missing required columns after mapping: {missing}")

for c in ["player_id","year","age","pa","H","1B","2B","3B","HR","BB","K","RBI","CS","SB","GIDP","SF","R","BA_csv","OBP_csv","SLG_csv","OPS_csv"]:
    if c in df.columns:
        df[c]=pd.to_numeric(df[c],errors="coerce")

df=df.dropna(subset=["player_id","year","age"]).copy()
df["player_id"]=df["player_id"].astype(int)
df["year"]=df["year"].astype(int)
df["age"]=df["age"].astype(int)

if "SF" not in df.columns:
    df["SF"]=0.0

df["ab"]=np.where(
    (df["BA_csv"]>0)&np.isfinite(df["BA_csv"])&np.isfinite(df["H"]),
    df["H"]/df["BA_csv"],
    np.nan
)
df["ab"]=df["ab"].fillna(df["pa"]-df["BB"]-df["SF"])
df["ab"]=pd.to_numeric(df["ab"],errors="coerce")
df["ab"]=df["ab"].where(df["ab"]>=0,np.nan)

df=df.sort_values(["player_id","year"]).reset_index(drop=True)

df["BB_per_AB"]=safe_div(df["BB"],df["ab"])
df["HR_per_AB"]=safe_div(df["HR"],df["ab"])
df["H_per_AB"]=safe_div(df["H"],df["ab"])
df["K_per_AB"]=safe_div(df["K"],df["ab"]) if "K" in df.columns else np.nan

counting_keep=["pa","ab","H","1B","2B","3B","HR","BB"]
extras=[c for c in ["K","RBI","SB","CS","GIDP","R","SF"] if c in df.columns]
rate_keep=["BB_per_AB","HR_per_AB","H_per_AB","K_per_AB"]
model_stats=[c for c in counting_keep+extras+rate_keep if c in df.columns]

age_avg=df.groupby("age")[model_stats].mean().reset_index().sort_values("age")
age_deltas=age_avg.copy()
for s in model_stats:
    age_deltas[f"{s}_age_delta"]=age_deltas[s].diff()
age_deltas=age_deltas[["age"]+[f"{s}_age_delta" for s in model_stats]].dropna()
next_age_deltas=age_deltas.rename(columns={"age":"next_age"})

for s in model_stats:
    df[f"{s}_next"]=df.groupby("player_id")[s].shift(-1)
train_df=df.dropna(subset=[f"{s}_next" for s in model_stats]).copy()

models={}
metrics_rows=[]
for s in model_stats:
    tmp=train_df[["player_id","year","age","pa",s,f"{s}_next"]].copy()
    tmp["next_age"]=tmp["age"]+1
    tmp=tmp.merge(next_age_deltas[["next_age",f"{s}_age_delta"]],on="next_age",how="left").dropna(subset=[f"{s}_age_delta"])
    if len(tmp)<MIN_ROWS_TO_TRAIN:
        continue
    X=tmp[[s,"age","pa",f"{s}_age_delta"]]
    y=tmp[f"{s}_next"]
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
    m=LinearRegression()
    m.fit(X_train,y_train)
    models[s]=m
    y_pred=m.predict(X_test)
    metrics_rows.append({"stat":s,"MAE":mean_absolute_error(y_test,y_pred),"R2":r2_score(y_test,y_pred),"n":len(tmp)})

model_metrics=pd.DataFrame(metrics_rows).sort_values("R2",ascending=False)

def project_next_year(pid:int):
    hist=df[df["player_id"]==pid].sort_values("year").copy()
    if hist.empty:
        return{},hist,pd.DataFrame(),pd.DataFrame()

    last=hist.iloc[-1]
    age_now=int(last["age"])
    age_next=age_now+1
    pa_val=to_scalar(last["pa"])

    age_only_counts={}
    reg_counts={}
    blend_counts={}

    for s in model_stats:
        current=to_scalar(last.get(s,np.nan))
        delta_series=age_deltas.loc[age_deltas["age"]==age_next,f"{s}_age_delta"]
        age_delta=to_scalar(delta_series) if len(delta_series) else np.nan

        age_only=current+age_delta if np.isfinite(current) and np.isfinite(age_delta) else np.nan

        reg=np.nan
        if (s in models) and np.isfinite(current) and np.isfinite(pa_val) and np.isfinite(age_delta):
            X_pred=build_X_pred(models[s],s,current,age_now,pa_val,age_delta)
            reg=float(models[s].predict(X_pred)[0])

        blend=0.5*age_only+0.5*reg if np.isfinite(age_only) and np.isfinite(reg) else np.nan

        age_only_counts[s]=age_only
        reg_counts[s]=reg
        blend_counts[s]=blend

    age_only_counts=ensure_1b_tb(age_only_counts)
    reg_counts=ensure_1b_tb(reg_counts)
    blend_counts=ensure_1b_tb(blend_counts)

    derived_age=compute_rates_from_projected_counts(age_only_counts)
    derived_reg=compute_rates_from_projected_counts(reg_counts)
    derived_blend=compute_rates_from_projected_counts(blend_counts)

    rows=[]
    proj={}
    for s in model_stats:
        proj[s]=blend_counts.get(s,np.nan)
        rows.append({"stat":s,"blend":blend_counts.get(s,np.nan),"age_only_proj":age_only_counts.get(s,np.nan),"regression_proj":reg_counts.get(s,np.nan)})

    for k in ["BA","OBP","SLG","OPS","TB"]:
        proj[k]=derived_blend.get(k,np.nan)
        rows.append({"stat":k,"blend":derived_blend.get(k,np.nan),"age_only_proj":derived_age.get(k,np.nan),"regression_proj":derived_reg.get(k,np.nan)})

    proj_table=pd.DataFrame(rows)
    for c in ["blend","age_only_proj","regression_proj"]:
        proj_table[c]=pd.to_numeric(proj_table[c],errors="coerce").round(4)

    blend_slash=format_triple(
        to_scalar(proj_table.loc[proj_table["stat"]=="BA","blend"]),
        to_scalar(proj_table.loc[proj_table["stat"]=="OBP","blend"]),
        to_scalar(proj_table.loc[proj_table["stat"]=="SLG","blend"])
    )
    age_slash=format_triple(
        to_scalar(proj_table.loc[proj_table["stat"]=="BA","age_only_proj"]),
        to_scalar(proj_table.loc[proj_table["stat"]=="OBP","age_only_proj"]),
        to_scalar(proj_table.loc[proj_table["stat"]=="SLG","age_only_proj"])
    )
    reg_slash=format_triple(
        to_scalar(proj_table.loc[proj_table["stat"]=="BA","regression_proj"]),
        to_scalar(proj_table.loc[proj_table["stat"]=="OBP","regression_proj"]),
        to_scalar(proj_table.loc[proj_table["stat"]=="SLG","regression_proj"])
    )

    triple_row=pd.DataFrame([{"stat":"TripleSlash","blend":blend_slash,"age_only_proj":age_slash,"regression_proj":reg_slash}])
    proj_table=pd.concat([proj_table,triple_row],ignore_index=True)

    order_top=proj_table[proj_table["stat"]!="TripleSlash"].sort_values("stat")
    order_bottom=proj_table[proj_table["stat"]=="TripleSlash"]
    proj_table=pd.concat([order_top,order_bottom],ignore_index=True)

    meta={"Name":last["Name"],"player_id":int(last["player_id"]),"last_year":int(last["year"]),"age_now":age_now,"age_next":age_next,"pa_last":pa_val}

    proj_wide={"Name":meta["Name"],"player_id":meta["player_id"],"age_next":meta["age_next"]}
    for k,v in proj.items():
        proj_wide[f"proj_{k}"]=v

    for c in list(proj_wide.keys()):
        if c.startswith("proj_"):
            v=proj_wide[c]
            if isinstance(v,(int,float,np.floating)) and np.isfinite(v):
                proj_wide[c]=round(float(v),4)
    if "proj_TB" in proj_wide and np.isfinite(proj_wide["proj_TB"]):
        proj_wide["proj_TB"]=round(float(proj_wide["proj_TB"]),0)

    proj_wide_df=pd.DataFrame([proj_wide])
    return meta,hist,proj_table,proj_wide_df

def build_all_projections(target_season=TARGET_SEASON,min_pa_last=0):
    last_rows=df.sort_values(["player_id","year"]).groupby("player_id").tail(1).copy()
    last_rows=last_rows[last_rows["pa"]>=min_pa_last]
    last_rows=last_rows[last_rows["year"]==target_season-1]
    out=[]
    for _,r in last_rows.iterrows():
        _,_,_,w=project_next_year(int(r["player_id"]))
        if w.empty:
            continue
        row=w.iloc[0].to_dict()
        row["proj_year"]=target_season
        out.append(row)
    return pd.DataFrame(out)

ALL_PROJ=build_all_projections(TARGET_SEASON,MIN_PA_FILTER_DEFAULT)
proj_cols=sorted([c for c in ALL_PROJ.columns if c.startswith("proj_")])

mode=widgets.ToggleButtons(options=["Player Lookup","Leaderboard"],value="Player Lookup",description="Mode:")
name_box=widgets.Text(value="",placeholder="Type full/partial name",description="Search:",layout=widgets.Layout(width="600px"))
player_dd=widgets.Dropdown(options=[],description="Matches:",layout=widgets.Layout(width="600px"))
stat_dd=widgets.Dropdown(options=proj_cols,description="Sort by:",layout=widgets.Layout(width="600px"))
order_dd=widgets.Dropdown(options=[("Highest","desc"),("Lowest","asc")],value="desc",description="Order:")
n_slider=widgets.IntSlider(value=WINDOW_N_FOR_LEADERBOARD,min=5,max=200,step=5,description="Show N:")
min_pa_box=widgets.IntText(value=MIN_PA_FILTER_DEFAULT,description="Min PA(last):")
rebuild_btn=widgets.Button(description="Rebuild",button_style="info")
out=widgets.Output()

def get_matches(q,limit=25):
    q=(q or "").strip().lower()
    if not q: return []
    matches=df[["Name","player_id"]].drop_duplicates()
    matches=matches[matches["Name"].str.lower().str.contains(q,na=False)].head(limit)
    return [(f"{row['Name']} (id={int(row['player_id'])})",int(row["player_id"])) for _,row in matches.iterrows()]

def refresh_player_matches(change=None):
    opts=get_matches(name_box.value)
    player_dd.options=opts
    if opts: player_dd.value=opts[0][1]

def clean_hist_for_display(hist):
    base=["Name","player_id","year","age","pa","ab"]
    stats=["H","1B","2B","3B","HR","BB","K","RBI","SB","CS","GIDP","R","SF"]
    rates=["BB_per_AB","HR_per_AB","H_per_AB","K_per_AB","BA_csv","OBP_csv","SLG_csv","OPS_csv"]
    keep=[c for c in base+stats+rates if c in hist.columns]
    out_hist=hist[keep].sort_values("year").copy()
    for c in ["BB_per_AB","HR_per_AB","H_per_AB","K_per_AB","BA_csv","OBP_csv","SLG_csv","OPS_csv"]:
        if c in out_hist.columns:
            out_hist[c]=pd.to_numeric(out_hist[c],errors="coerce").round(4)
    if "ab" in out_hist.columns:
        out_hist["ab"]=pd.to_numeric(out_hist["ab"],errors="coerce").round(0)
    return out_hist

def rebuild(btn=None):
    global ALL_PROJ,proj_cols
    with out:
        clear_output(wait=True)
        print(f"Rebuilding {TARGET_SEASON} leaderboard...")
    ALL_PROJ=build_all_projections(TARGET_SEASON,int(min_pa_box.value))
    proj_cols=sorted([c for c in ALL_PROJ.columns if c.startswith("proj_")])
    stat_dd.options=proj_cols
    if proj_cols: stat_dd.value=proj_cols[0]
    render()

def render():
    with out:
        clear_output(wait=True)
        if mode.value=="Player Lookup":
            if player_dd.value is None:
                print("Type a name to search.")
                return

            meta,hist,proj_table,proj_wide_df=project_next_year(int(player_dd.value))
            print(meta)

            print("\n---Last Year Snapshot---")
            display(clean_hist_for_display(hist).tail(1))

            print("\n---Full Prior Years---")
            display(clean_hist_for_display(hist))

            print("\n---Next Year Projections (blend)---")
            display(proj_table.style.format({"blend":fmt_cell,"age_only_proj":fmt_cell,"regression_proj":fmt_cell},na_rep=""))

            print(f"\n---{TARGET_SEASON} Projection Card (50/50 blend)---")
            show_cols=["Name","player_id","age_next","proj_BA","proj_OBP","proj_SLG","proj_OPS"]
            show_cols=[c for c in show_cols if c in proj_wide_df.columns]
            display(proj_wide_df[show_cols])

            print("\n---Model quality---")
            display(model_metrics)

        else:
            if stat_dd.value is None:
                print("No proj_* columns available.")
                return
            col=stat_dd.value
            asc=(order_dd.value=="asc")
            n=int(n_slider.value)
            board=ALL_PROJ[["Name","player_id","proj_year",col]].dropna(subset=[col]).sort_values(col,ascending=asc).head(n)
            print(f"{TARGET_SEASON} Leaderboard sorted by: {col} ({'Lowest' if asc else 'Highest'})")
            display(board)

name_box.observe(refresh_player_matches,names="value")
player_dd.observe(lambda change:render(),names="value")
mode.observe(lambda change:render(),names="value")
stat_dd.observe(lambda change:render(),names="value")
order_dd.observe(lambda change:render(),names="value")
n_slider.observe(lambda change:render(),names="value")
rebuild_btn.on_click(rebuild)

display(widgets.VBox([mode,name_box,player_dd,stat_dd,widgets.HBox([order_dd,n_slider]),widgets.HBox([min_pa_box,rebuild_btn]),out]))
render()


Pitchers

In [1]:
# =========================
# Pitcher Projection Block
# (matches the hitter notebook style, but uses stats (29).csv)
# =========================

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import ipywidgets as widgets
from IPython.display import display, clear_output

TARGET_SEASON = 2026
WINDOW_N_FOR_LEADERBOARD = 50
MIN_BF_FILTER_DEFAULT = 0          # BF = batters faced (your CSV column is "pa")
MIN_ROWS_TO_TRAIN = 50

def to_scalar(x):
    if isinstance(x, pd.Series): x = x.iloc[0]
    x = pd.to_numeric(x, errors="coerce")
    return float(x) if pd.notna(x) else np.nan

def safe_div(num, den):
    num = pd.to_numeric(num, errors="coerce")
    den = pd.to_numeric(den, errors="coerce")
    return np.where((den > 0) & np.isfinite(den) & np.isfinite(num), num / den, np.nan)

def build_name(df):
    if "last_name, first_name" in df.columns:
        df["Name"] = df["last_name, first_name"].astype(str)
        return df
    if "last_name" in df.columns and "first_name" in df.columns:
        df["Name"] = df["last_name"].astype(str) + ", " + df["first_name"].astype(str)
        return df
    if "Name" in df.columns:
        return df
    raise ValueError("Missing name column. Need 'last_name, first_name' or Name or (first_name+last_name).")

def ip_to_outs(ip_series):
    """
    Converts baseball innings formatted like 5.1, 5.2 into OUTS.
    5.0 -> 15 outs, 5.1 -> 16 outs, 5.2 -> 17 outs
    """
    x = pd.to_numeric(ip_series, errors="coerce")
    whole = np.floor(x).astype("Int64")
    frac = np.round((x - np.floor(x)) * 10).astype("Int64")  # 0,1,2 (from .0/.1/.2)
    frac = frac.where(frac.isin([0, 1, 2]), other=pd.NA)
    outs = whole * 3 + frac
    return pd.to_numeric(outs, errors="coerce")

def outs_to_ip(outs):
    outs = pd.to_numeric(outs, errors="coerce")
    return outs / 3.0

def map_pitcher_columns(df_in):
    d = df_in.copy()
    rename = {
        "player_age": "age",
        "pa": "bf",                    # batters faced
        "p_formatted_ip": "IP_fmt",
        "p_earned_run": "ER",
        "p_run": "R",
        "p_win": "W",
        "p_loss": "L",
        "p_era": "ERA_csv",
        "p_quality_start": "QS",
        "p_hit_by_pitch": "HBP",
        # these are against (allowed):
        "hit": "H",
        "home_run": "HR",
        "strikeout": "K",
        "walk": "BB",
        # optional (against slash):
        "batting_avg": "BAA_csv",
        "on_base_percent": "OBA_csv",
        "slg_percent": "SLGA_csv",
        "on_base_plus_slg": "OPSA_csv",
        "babip": "BABIP_csv",
        "p_inh_runner_scored": "IRS",
    }
    d = d.rename(columns={k: v for k, v in rename.items() if k in d.columns})
    return d

def build_X_pred(model, stat_name, current, age_now, bf_val, age_delta):
    cols = list(model.feature_names_in_)
    row = {}
    for c in cols:
        if c == stat_name: row[c] = current
        elif c == "age": row[c] = age_now
        elif c == "bf": row[c] = bf_val
        elif c == f"{stat_name}_age_delta": row[c] = age_delta
        else: row[c] = np.nan
    return pd.DataFrame([row], columns=cols)

def fmt_cell(x):
    if isinstance(x, str): return x
    if pd.isna(x): return ""
    try:
        return f"{float(x):.4f}"
    except Exception:
        return str(x)

def compute_derived_pitching(p):
    # p contains projected counts like outs, ER, H, BB, K, HR
    outs = to_scalar(p.get("ip_outs", np.nan))
    ip = outs_to_ip(outs) if np.isfinite(outs) and outs > 0 else np.nan
    ER = to_scalar(p.get("ER", np.nan))
    H  = to_scalar(p.get("H", np.nan))
    BB = to_scalar(p.get("BB", np.nan))
    K  = to_scalar(p.get("K", np.nan))
    HR = to_scalar(p.get("HR", np.nan))

    ERA  = (ER * 9 / ip) if np.isfinite(ER) and np.isfinite(ip) and ip > 0 else np.nan
    WHIP = ((BB + H) / ip) if np.isfinite(BB) and np.isfinite(H) and np.isfinite(ip) and ip > 0 else np.nan
    K9   = (K * 9 / ip) if np.isfinite(K) and np.isfinite(ip) and ip > 0 else np.nan
    BB9  = (BB * 9 / ip) if np.isfinite(BB) and np.isfinite(ip) and ip > 0 else np.nan
    HR9  = (HR * 9 / ip) if np.isfinite(HR) and np.isfinite(ip) and ip > 0 else np.nan

    return {"IP": ip, "ERA": ERA, "WHIP": WHIP, "K9": K9, "BB9": BB9, "HR9": HR9}

# ---- Load CSV (this is YOUR pitcher CSV) ----
df = pd.read_csv("stats (29).csv").copy()
df = df.loc[:, ~df.columns.duplicated()].copy()
df = build_name(df)
df = map_pitcher_columns(df)

required = {"Name", "player_id", "year", "age", "bf", "IP_fmt", "ER", "H", "HR", "BB", "K", "ERA_csv"}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns after mapping: {missing}")

# numeric cleanup
num_cols = [
    "player_id","year","age","bf","IP_fmt","ER","R","W","L","QS","HBP","IRS",
    "H","HR","BB","K","ERA_csv","BAA_csv","OBA_csv","SLGA_csv","OPSA_csv","BABIP_csv"
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

df = df.dropna(subset=["player_id","year","age","bf","IP_fmt"]).copy()
df["player_id"] = df["player_id"].astype(int)
df["year"] = df["year"].astype(int)
df["age"] = df["age"].astype(int)

# innings -> outs + decimal IP
df["ip_outs"] = ip_to_outs(df["IP_fmt"])
df["IP"] = outs_to_ip(df["ip_outs"])
df = df.dropna(subset=["ip_outs"]).copy()

df = df.sort_values(["player_id","year"]).reset_index(drop=True)

# per BF rates (analogous to hitter per AB rates)
df["K_per_BF"]  = safe_div(df["K"],  df["bf"])
df["BB_per_BF"] = safe_div(df["BB"], df["bf"])
df["HR_per_BF"] = safe_div(df["HR"], df["bf"])
df["H_per_BF"]  = safe_div(df["H"],  df["bf"])

# model stats to project (counts + rates)
counting_keep = ["bf","ip_outs","ER","H","HR","BB","K"]
extras = [c for c in ["W","L","QS","HBP","IRS","R"] if c in df.columns]
rate_keep = ["K_per_BF","BB_per_BF","HR_per_BF","H_per_BF"]
model_stats = [c for c in counting_keep + extras + rate_keep if c in df.columns]

# age averages + age deltas (same idea as hitter block)
age_avg = df.groupby("age")[model_stats].mean().reset_index().sort_values("age")
age_deltas = age_avg.copy()
for s in model_stats:
    age_deltas[f"{s}_age_delta"] = age_deltas[s].diff()
age_deltas = age_deltas[["age"] + [f"{s}_age_delta" for s in model_stats]].dropna()
next_age_deltas = age_deltas.rename(columns={"age":"next_age"})

# build next-year targets
for s in model_stats:
    df[f"{s}_next"] = df.groupby("player_id")[s].shift(-1)
train_df = df.dropna(subset=[f"{s}_next" for s in model_stats]).copy()

# train per-stat linear regressions
models = {}
metrics_rows = []
for s in model_stats:
    tmp = train_df[["player_id","year","age","bf",s,f"{s}_next"]].copy()
    tmp["next_age"] = tmp["age"] + 1
    tmp = tmp.merge(next_age_deltas[["next_age", f"{s}_age_delta"]], on="next_age", how="left") \
             .dropna(subset=[f"{s}_age_delta"])
    if len(tmp) < MIN_ROWS_TO_TRAIN:
        continue

    X = tmp[[s, "age", "bf", f"{s}_age_delta"]]
    y = tmp[f"{s}_next"]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    m = LinearRegression()
    m.fit(X_train, y_train)

    models[s] = m
    y_pred = m.predict(X_test)
    metrics_rows.append({
        "stat": s,
        "MAE": mean_absolute_error(y_test, y_pred),
        "R2": r2_score(y_test, y_pred),
        "n": len(tmp),
    })

model_metrics = pd.DataFrame(metrics_rows).sort_values("R2", ascending=False)

def project_next_year(pid: int):
    hist = df[df["player_id"] == pid].sort_values("year").copy()
    if hist.empty:
        return {}, hist, pd.DataFrame(), pd.DataFrame()

    last = hist.iloc[-1]
    age_now = int(last["age"])
    age_next = age_now + 1
    bf_val = to_scalar(last["bf"])

    age_only = {}
    reg = {}
    blend = {}

    for s in model_stats:
        current = to_scalar(last.get(s, np.nan))
        delta_series = age_deltas.loc[age_deltas["age"] == age_next, f"{s}_age_delta"]
        age_delta = to_scalar(delta_series) if len(delta_series) else np.nan

        a = current + age_delta if np.isfinite(current) and np.isfinite(age_delta) else np.nan

        r = np.nan
        if (s in models) and np.isfinite(current) and np.isfinite(bf_val) and np.isfinite(age_delta):
            X_pred = build_X_pred(models[s], s, current, age_now, bf_val, age_delta)
            r = float(models[s].predict(X_pred)[0])

        b = 0.5 * a + 0.5 * r if np.isfinite(a) and np.isfinite(r) else np.nan

        age_only[s] = a
        reg[s] = r
        blend[s] = b

    # derived metrics from blended projection
    derived_age = compute_derived_pitching(age_only)
    derived_reg = compute_derived_pitching(reg)
    derived_blend = compute_derived_pitching(blend)

    rows = []
    proj = {}

    for s in model_stats:
        proj[s] = blend.get(s, np.nan)
        rows.append({"stat": s, "blend": blend.get(s, np.nan), "age_only_proj": age_only.get(s, np.nan), "regression_proj": reg.get(s, np.nan)})

    for k in ["IP","ERA","WHIP","K9","BB9","HR9"]:
        proj[k] = derived_blend.get(k, np.nan)
        rows.append({"stat": k, "blend": derived_blend.get(k, np.nan), "age_only_proj": derived_age.get(k, np.nan), "regression_proj": derived_reg.get(k, np.nan)})

    proj_table = pd.DataFrame(rows)
    for c in ["blend","age_only_proj","regression_proj"]:
        proj_table[c] = pd.to_numeric(proj_table[c], errors="coerce").round(4)

    meta = {
        "Name": last["Name"],
        "player_id": int(last["player_id"]),
        "last_year": int(last["year"]),
        "age_now": age_now,
        "age_next": age_next,
        "bf_last": bf_val,
        "ip_last": to_scalar(last.get("IP", np.nan)),
        "era_last": to_scalar(last.get("ERA_csv", np.nan)),
    }

    proj_wide = {"Name": meta["Name"], "player_id": meta["player_id"], "age_next": meta["age_next"]}
    for k, v in proj.items():
        proj_wide[f"proj_{k}"] = v

    # rounding nicer for display
    for c in list(proj_wide.keys()):
        if c.startswith("proj_"):
            v = proj_wide[c]
            if isinstance(v, (int, float, np.floating)) and np.isfinite(v):
                proj_wide[c] = round(float(v), 4)

    # show IP as 1 decimal, outs as integer (if present)
    if "proj_ip_outs" in proj_wide and np.isfinite(proj_wide["proj_ip_outs"]):
        proj_wide["proj_ip_outs"] = int(round(float(proj_wide["proj_ip_outs"])))
    if "proj_IP" in proj_wide and np.isfinite(proj_wide["proj_IP"]):
        proj_wide["proj_IP"] = round(float(proj_wide["proj_IP"]), 1)

    proj_wide_df = pd.DataFrame([proj_wide])
    return meta, hist, proj_table, proj_wide_df

def build_all_projections(target_season=TARGET_SEASON, min_bf_last=0):
    last_rows = df.sort_values(["player_id","year"]).groupby("player_id").tail(1).copy()
    last_rows = last_rows[(last_rows["bf"] >= min_bf_last) & (last_rows["year"] == target_season - 1)]
    out = []
    for _, r in last_rows.iterrows():
        _, _, _, w = project_next_year(int(r["player_id"]))
        if w.empty:
            continue
        row = w.iloc[0].to_dict()
        row["proj_year"] = target_season
        out.append(row)
    return pd.DataFrame(out)

ALL_PROJ = build_all_projections(TARGET_SEASON, MIN_BF_FILTER_DEFAULT)
proj_cols = sorted([c for c in ALL_PROJ.columns if c.startswith("proj_")])

# ---- Widgets (same UX as hitters) ----
mode = widgets.ToggleButtons(options=["Player Lookup", "Leaderboard"], value="Player Lookup", description="Mode:")
name_box = widgets.Text(value="", placeholder="Type full/partial name", description="Search:", layout=widgets.Layout(width="600px"))
player_dd = widgets.Dropdown(options=[], description="Matches:", layout=widgets.Layout(width="600px"))
stat_dd = widgets.Dropdown(options=proj_cols, description="Sort by:", layout=widgets.Layout(width="600px"))
order_dd = widgets.Dropdown(options=[("Lowest", "asc"), ("Highest", "desc")], value="asc", description="Order:")
n_slider = widgets.IntSlider(value=WINDOW_N_FOR_LEADERBOARD, min=5, max=200, step=5, description="Show N:")
min_bf_box = widgets.IntText(value=MIN_BF_FILTER_DEFAULT, description="Min BF(last):")
rebuild_btn = widgets.Button(description="Rebuild", button_style="info")
out = widgets.Output()

def get_matches(q, limit=25):
    q = (q or "").strip().lower()
    if not q: return []
    matches = df[["Name","player_id"]].drop_duplicates()
    matches = matches[matches["Name"].str.lower().str.contains(q, na=False)].head(limit)
    return [(f"{row['Name']} (id={int(row['player_id'])})", int(row["player_id"])) for _, row in matches.iterrows()]

def refresh_player_matches(change=None):
    opts = get_matches(name_box.value)
    player_dd.options = opts
    if opts: player_dd.value = opts[0][1]

def clean_hist_for_display(hist):
    base = ["Name","player_id","year","age","bf","IP","ip_outs","ERA_csv"]
    stats = ["ER","H","HR","BB","K","W","L","QS","HBP","IRS","R"]
    rates = ["K_per_BF","BB_per_BF","HR_per_BF","H_per_BF","BAA_csv","OBA_csv","SLGA_csv","OPSA_csv","BABIP_csv"]
    keep = [c for c in base + stats + rates if c in hist.columns]
    out_hist = hist[keep].sort_values("year").copy()

    for c in ["ERA_csv","K_per_BF","BB_per_BF","HR_per_BF","H_per_BF","BAA_csv","OBA_csv","SLGA_csv","OPSA_csv","BABIP_csv"]:
        if c in out_hist.columns:
            out_hist[c] = pd.to_numeric(out_hist[c], errors="coerce").round(4)
    if "IP" in out_hist.columns:
        out_hist["IP"] = pd.to_numeric(out_hist["IP"], errors="coerce").round(1)
    if "ip_outs" in out_hist.columns:
        out_hist["ip_outs"] = pd.to_numeric(out_hist["ip_outs"], errors="coerce").round(0)

    return out_hist

def rebuild(btn=None):
    global ALL_PROJ, proj_cols
    with out:
        clear_output(wait=True)
        print(f"Rebuilding {TARGET_SEASON} pitcher leaderboard...")
    ALL_PROJ = build_all_projections(TARGET_SEASON, int(min_bf_box.value))
    proj_cols = sorted([c for c in ALL_PROJ.columns if c.startswith("proj_")])
    stat_dd.options = proj_cols
    if proj_cols: stat_dd.value = proj_cols[0]
    render()

def render():
    with out:
        clear_output(wait=True)
        if mode.value == "Player Lookup":
            if player_dd.value is None:
                print("Type a name to search.")
                return

            meta, hist, proj_table, proj_wide_df = project_next_year(int(player_dd.value))
            print(meta)

            print("\n---Last Year Snapshot---")
            display(clean_hist_for_display(hist).tail(1))

            print("\n---Full Prior Years---")
            display(clean_hist_for_display(hist))

            print("\n---Next Year Projections (blend)---")
            display(proj_table.style.format({"blend": fmt_cell, "age_only_proj": fmt_cell, "regression_proj": fmt_cell}, na_rep=""))

            print(f"\n---{TARGET_SEASON} Projection Card (50/50 blend)---")
            show_cols = ["Name","player_id","age_next","proj_IP","proj_ERA","proj_WHIP","proj_K9","proj_BB9","proj_HR9"]
            show_cols = [c for c in show_cols if c in proj_wide_df.columns]
            display(proj_wide_df[show_cols])

            print("\n---Model quality---")
            display(model_metrics)

        else:
            if stat_dd.value is None:
                print("No proj_* columns available.")
                return
            col = stat_dd.value
            asc = (order_dd.value == "asc")
            n = int(n_slider.value)
            board = ALL_PROJ[["Name","player_id","proj_year",col]].dropna(subset=[col]).sort_values(col, ascending=asc).head(n)
            print(f"{TARGET_SEASON} Pitcher Leaderboard sorted by: {col} ({'Lowest' if asc else 'Highest'})")
            display(board)

name_box.observe(refresh_player_matches, names="value")
player_dd.observe(lambda change: render(), names="value")
mode.observe(lambda change: render(), names="value")
stat_dd.observe(lambda change: render(), names="value")
order_dd.observe(lambda change: render(), names="value")
n_slider.observe(lambda change: render(), names="value")
rebuild_btn.on_click(rebuild)

display(widgets.VBox([
    mode,
    name_box,
    player_dd,
    stat_dd,
    widgets.HBox([order_dd, n_slider]),
    widgets.HBox([min_bf_box, rebuild_btn]),
    out
]))
render()
